In [78]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'html.parser')
table = soup.find_all('table')[0]
trs = table.find_all('tr')[1:]
for i in range(0,len(trs)):
    tds = trs[i].text.split('\n')
    if tds[2] == 'Not assigned':
        continue
    postalCode = tds[1]
    borough = tds[2]
    neighborhood = tds[3]
    if neighborhood == 'Not assigned':
        neighborhood = borough
    if len(df) != 0:
        if df.iloc[len(df)-1].PostalCode == tds[1] :
            tmp = 'test'
            df.loc[len(df)-1,['Neighborhood']] = df.iloc[len(df)-1].Neighborhood + ", " + neighborhood
            continue
    df = df.append({'PostalCode': postalCode,
                    'Borough': borough,
                    'Neighborhood': neighborhood}, ignore_index=True)
df['Latitude'] = np.nan
df['Longitude'] = np.nan

geo = pd.read_csv('http://cocl.us/Geospatial_data')

for index in range(0, df['PostalCode'].shape[0]):
    postal_indexes = geo.loc[geo['Postal Code'] == df['PostalCode'].iloc[index]]
    if (postal_indexes.index.size != 0):
        df.loc[index, 'Latitude'] = float(format(geo['Latitude'].iloc[postal_indexes.index[0]],'.7f'))
        df.loc[index, 'Longitude'] = float(format(geo['Longitude'].iloc[postal_indexes.index[0]],'.7f'))
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [29]:
df.shape

(103, 5)

In [81]:
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

kclusters = 5

df_clustering = df[['Latitude','Longitude']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustering)
df['ClusterLabels'] = kmeans.labels_

# create map
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters